In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import datetime as dt
import numpy as np
from fuzzywuzzy import process
import time

In [2]:
mwbe = pd.read_csv('M_WBE__LBE__and_EBE_Certified_Business_List.csv' )
nycorp = pd.read_csv("Active_Corporations___Beginning_1800.csv")

In [3]:
mwbe.head()

,Vendor_Formal_Name,Vendor_DBA,Contact_Name,telephone,fax,Email,certification,Cert_Renewal_Date,Ethnicity,Address1,...,Job_Exp3_Name_of_Client,Job_Exp3_Value_of_Contract,Job_Exp3_Percent_Self_Performed,DATEOFJOB3,EXPERIENCE3,Job_Exp4_Name_of_Client,Job_Exp4_Value_of_Contract,Job_Exp4_Percent_Self_Performed,DATEOFJOB4,EXPERIENCE4
0,Bryce Construction Corp.,NaN,Patrick Nwokeji,6465235353,9736786112,bryce.constructioncorp@gmail.com,MBE,06/30/2024,BLACK,3121 Kingsland Avenue,...,NYC Dept. of Parks & Recreation,207000.0,NaN,04/01/2009,Windows replacement wall & ceiling framing & r...,NYC Dept. of Parks & Recreation,207000.0,NaN,04/01/2009,Windows replacement wall & ceiling framing & r...
1,iCommercialize LLC,NaN,Beth Hirschhorn,9177348658,NaN,beth.hirschhorn@icommercialize.co,WBE,NaN,NON-MINORITY,305 East 24th Street,...,Namecheap Inc,NaN,100.0,08/30/2016,"Conduct customer and competitive analysis, mar...",NaN,NaN,NaN,NaN,NaN
2,Slice Wireless Solutions LLC,NaN,Aleksandr Yakubov,2128686900,6464488238,NaN,WBE,NaN,NON-MINORITY,347 West 36th Street,...,Action Without Borders,36000.0,100.0,07/10/2015,Manage services for action without borders net...,NaN,NaN,NaN,NaN,NaN
3,"Alpha and Omega Consulting Group, Inc",NaN,Cinque Braithwaite,9172840028,NaN,mpericles1@yahoo.com,"MBE,WBE",02/27/2020;02/27/2020,BLACK,219-08 119th Ave,...,Sirius XM,NaN,20.0,01/01/2013,1. Lead the transition of the Sirius xm Canada...,NaN,NaN,NaN,NaN,NaN
4,Bay Ridge Security Service Inc.,NaN,Anthony La Bella,7182382727,7188363474,nl@bayridgesecurity.com,WBE,NaN,NON-MINORITY,110 Bay Ridge Avenue,...,Medspaw Associates,59560.0,100.0,09/01/2017,Discourage loiterers and shoplifters.,NaN,NaN,NaN,NaN,NaN


In [59]:
nycorp.head(5)

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent City,Registered Agent State,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,DOS Address
32,5309183,0113 FOOD CORP.,2018-03-22,BRONX,NEW YORK,DOMESTIC BUSINESS CORPORATION,0113 FOOD CORP.,184 W. 231ST STREET,,BRONX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184 W. 231ST STREET
45,5383113,01 ENVIRONMENTAL SERVICES INC,2018-07-26,NEW YORK,NEW YORK,DOMESTIC BUSINESS CORPORATION,01 ENVIRONMENTAL SERVICES INC,57 WEST 57TH ST.,STE 414,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57 WEST 57TH ST. STE 414
48,5374317,"01 WEST AVENUE, LLC",2018-07-12,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"NATIONAL REGISTERED AGENTS, INC.",28 LIBERTY ST.,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28 LIBERTY ST.
49,5271749,"0213EIGHT, LLC",2018-01-23,ALBANY,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"0213EIGHT, LLC",30 FIELDS END,,GLENMONT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 FIELDS END
52,5298916,022218 LLC,2018-03-06,KINGS,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,022218 LLC,"5014 16TH AVENUE, SUITE 462",,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5014 16TH AVENUE, SUITE 462"


In [5]:
print('mwbe has {} rows, {} columns'.format(mwbe.shape[0],mwbe.shape[1]))
print('nycorp has {} rows, {} columns'.format(nycorp.shape[0],nycorp.shape[1]))

mwbe has 9036 rows, 46 columns
nycorp has 2721217 rows, 30 columns


### Turn date of establishment and DOS filing date to date-time

In [6]:
nycorp['Initial DOS Filing Date'] = pd.to_datetime(nycorp['Initial DOS Filing Date'], errors = 'coerce')

In [7]:
#change certified into date time
mwbe['Date_Of_Establishment'] = pd.to_datetime(mwbe['Date_Of_Establishment'], errors = 'coerce')

* Get the companies that were established in the year 2018

In [8]:
#drop the rows of any establishment that is not 2018 to get only 2018
mwbe.drop(mwbe[mwbe['Date_Of_Establishment'].dt.year != 2018 ].index, inplace = True)

In [9]:
#drop the rows of any corp that is not 2018 to get only 2018
nycorp.drop(nycorp[nycorp['Initial DOS Filing Date'].dt.year != 2018].index,inplace = True)

* The amount of values is significantly less for both mwbe and nycorp

In [10]:
print(mwbe.shape)
print(nycorp.shape)

(127, 46)
(199544, 30)


### Fill null values in dos address 1,2 and address2,address1.
* address1 did not have any null values 

In [11]:
nycorp['DOS Process Address 1'].fillna('', inplace = True)

In [12]:
nycorp['DOS Process Address 2'].fillna('',inplace = True)

In [13]:
mwbe['Address2'].fillna('', inplace = True)

add together DOS Proces Address 1 & 2 | add together Address 1 & 2

In [14]:
nycorp['DOS Address'] = nycorp['DOS Process Address 1']  + ' ' + nycorp['DOS Process Address 2']

In [15]:
mwbe['MWBE Address'] = mwbe['Address1'] + ' ' + mwbe['Address2']

apply upper to mwbe to match the nycorps column

In [16]:
mwbe['MWBE Address'] = mwbe['MWBE Address'].apply(lambda x: x.upper())

In [17]:
mwbe['Vendor_Formal_Name'] = mwbe['Vendor_Formal_Name'].apply(lambda x: x.upper())

In [61]:
nycorp[['DOS Process Address 1','DOS Process Address 2', 'DOS Address']]

,DOS Process Address 1,DOS Process Address 2,DOS Address
32,184 W. 231ST STREET,,184 W. 231ST STREET
45,57 WEST 57TH ST.,STE 414,57 WEST 57TH ST. STE 414
48,28 LIBERTY ST.,,28 LIBERTY ST.
49,30 FIELDS END,,30 FIELDS END
52,"5014 16TH AVENUE, SUITE 462",,"5014 16TH AVENUE, SUITE 462"
57,812 58TH STREET,,812 58TH STREET
61,620 EAST 186 STREET,,620 EAST 186 STREET
71,366 E 204TH STREET,,366 E 204TH STREET
84,43 GRAMATAN AVE,,43 GRAMATAN AVE
90,"405 PARK AVENUE, SUITE 500",,"405 PARK AVENUE, SUITE 500"


### Look at the data now
* notice the extra column in both nycorp and mwbe

In [18]:
nycorp.head(1)

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent City,Registered Agent State,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,DOS Address
32,5309183,0113 FOOD CORP.,2018-03-22,BRONX,NEW YORK,DOMESTIC BUSINESS CORPORATION,0113 FOOD CORP.,184 W. 231ST STREET,,BRONX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184 W. 231ST STREET


In [19]:
mwbe.head(1)

,Vendor_Formal_Name,Vendor_DBA,Contact_Name,telephone,fax,Email,certification,Cert_Renewal_Date,Ethnicity,Address1,...,Job_Exp3_Value_of_Contract,Job_Exp3_Percent_Self_Performed,DATEOFJOB3,EXPERIENCE3,Job_Exp4_Name_of_Client,Job_Exp4_Value_of_Contract,Job_Exp4_Percent_Self_Performed,DATEOFJOB4,EXPERIENCE4,MWBE Address
78,NEW ROC CONTRACTING CORP.,NaN,Bojana Cfarku,914-500-3994,NaN,bcfarku@newroccontracting.com,WBE,NaN,NON-MINORITY,102 Jericho Turnpike,...,7200000.0,0.0,10/26/2016,masonry restoration of 8 nine story buildings ...,NaN,NaN,NaN,NaN,NaN,102 JERICHO TURNPIKE SUITE 102


### fuzzymatch the DOS Address and MWBE Address columns and put the values in a list

In [64]:
print(nycorp.shape)
print(mwbe.shape)

(199544, 31)
(127, 47)


In [20]:
start = time.time()
matched_cols = []

for row in nycorp.index:
    nycorp_add = nycorp.get_value(row,"DOS Address")
    for i in mwbe.index:
        mwbe_add =mwbe.get_value(i,"MWBE Address")
        matched_token=fuzz.token_set_ratio(nycorp_add,mwbe_add)
        if matched_token> 95:
            matched_cols.append([nycorp_add,mwbe_add,matched_token])
            
end = time.time()
time_taken = end - start
print('Time: ',time_taken)

C:\Users\jshyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
C:\Users\jshyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


Time:  596.0415298938751


### Notice 
* There are some repeated values 
* Some values do not match, such as the first value in the list

In [21]:
matched_cols

[['#122 ', '122 EGBERT AVE # 2', 100],
 ['950 THIRD AVENUE ', '950 THIRD AVENUE 24TH FLOOR', 100],
 ['950 THIRD AVENUE ', '950 THIRD AVENUE 24TH FLOOR', 100],
 ['89 SUMMIT STREET ', '9 SUMMIT STREET ', 97],
 ['370 LEXINGTON AVENUE 27TH FLOOR', '370 LEXINGTON AVENUE ', 100],
 ['370 LEXINGTON AVENUE, SUITE # 800', '370 LEXINGTON AVENUE ', 100],
 ['950 THIRD AVENUE 27TH FLOOR', '950 THIRD AVENUE 24TH FLOOR', 96],
 ['200 WEST STREET ', '200 WEST 54TH STREET, APT. 7J ', 100],
 ['200 WEST STREET ', '200 WEST 54TH STREET, APT. 7J ', 100],
 ['75 BROAD STREET ', '75 BROAD STREET, SUITE 2110 ', 100],
 ['120 ELGAR PLACE APT 26 L', '120 ELGAR PLACE ', 100],
 ['44 COURT STREET SUITE 1217 ', '44 COURT STREET STE 1217 ', 96],
 ['454 MANHATTAN AVE APT 5G', '454 MANHATTAN AVE ', 100],
 ['222 BROADWAY ', '222 BROADWAY,F19TH ', 100],
 ['370 LEXINGTON AVENUE, STE. 908 ', '370 LEXINGTON AVENUE ', 100],
 ['44 COURT STREET, SUITE 1217 ', '44 COURT STREET STE 1217 ', 96],
 ['44 COURT STREET ', '44 COURT STREE

In [45]:
nycorp.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent City,Registered Agent State,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,Address Concatenated
32,5309183,0113 FOOD CORP.,2018-03-22,BRONX,NEW YORK,DOMESTIC BUSINESS CORPORATION,0113 FOOD CORP.,184 W. 231ST STREET,,BRONX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184 W. 231ST STREET
45,5383113,01 ENVIRONMENTAL SERVICES INC,2018-07-26,NEW YORK,NEW YORK,DOMESTIC BUSINESS CORPORATION,01 ENVIRONMENTAL SERVICES INC,57 WEST 57TH ST.,STE 414,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57 WEST 57TH ST. STE 414
48,5374317,"01 WEST AVENUE, LLC",2018-07-12,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"NATIONAL REGISTERED AGENTS, INC.",28 LIBERTY ST.,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28 LIBERTY ST.
49,5271749,"0213EIGHT, LLC",2018-01-23,ALBANY,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"0213EIGHT, LLC",30 FIELDS END,,GLENMONT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 FIELDS END
52,5298916,022218 LLC,2018-03-06,KINGS,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,022218 LLC,"5014 16TH AVENUE, SUITE 462",,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5014 16TH AVENUE, SUITE 462"


### Create new dataframe for both nycorp & mwbe where the rows match their respective 'matched_cols'
* The first value in matched_cols came from nycorps
* The second value in matched_cols came from mwbe
* The third value is the matched_score

In [22]:
nycorp_matched = nycorp[nycorp['DOS Address'].isin(m[0] for m in matched_cols)]

In [23]:
mwbe_matched = mwbe[mwbe['MWBE Address'].isin(m[1] for m in matched_cols)]

In [24]:
mwbe_matched

,Vendor_Formal_Name,Vendor_DBA,Contact_Name,telephone,fax,Email,certification,Cert_Renewal_Date,Ethnicity,Address1,...,Job_Exp3_Value_of_Contract,Job_Exp3_Percent_Self_Performed,DATEOFJOB3,EXPERIENCE3,Job_Exp4_Name_of_Client,Job_Exp4_Value_of_Contract,Job_Exp4_Percent_Self_Performed,DATEOFJOB4,EXPERIENCE4,MWBE Address
78,NEW ROC CONTRACTING CORP.,NaN,Bojana Cfarku,914-500-3994,NaN,bcfarku@newroccontracting.com,WBE,NaN,NON-MINORITY,102 Jericho Turnpike,...,7200000.00,0.0,10/26/2016,masonry restoration of 8 nine story buildings ...,NaN,NaN,NaN,NaN,NaN,102 JERICHO TURNPIKE SUITE 102
184,TRIUMPH PRODUCTIONS INC.,NaN,Anne Pavone,2125991366,2126976460,triumphprod@mindspring.com,WBE,NaN,NON-MINORITY,370 Lexington Avenue,...,4450.00,100.0,01/01/2018,Printing,NaN,NaN,NaN,NaN,NaN,370 LEXINGTON AVENUE
234,"JC PROTECTION, LLC",NaN,clifford strong,646-335-6162,NaN,jcprotectionllc@gmail.com,MBE,06/30/2024,BLACK,26 Kermit Ave,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26 KERMIT AVE
306,"HUT CONSULTING, LLC",NaN,Harumi Urata-Thompson,646-271-0003,NaN,harumi@hutconsulting.com,"MBE,WBE",06/30/2024;06/30/2024,ASIAN,"200 West 54th Street, Apt. 7J",...,4875.00,100.0,02/04/2018,the client is moving from Australia to open a ...,NaN,NaN,NaN,NaN,NaN,"200 WEST 54TH STREET, APT. 7J"
656,EXCEED FLOORING INC,NaN,Mirela Niculescu,516-787-4885,NaN,exceedflooring@gmail.com,WBE,NaN,NON-MINORITY,74 Irma Drive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 IRMA DRIVE
893,HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.,NaN,Larry Zweigbaum,212-652-2966,212-691-7088,info@heintges.com,WBE,NaN,NON-MINORITY,"440 Park Avenue South, 15th Floor",...,0.00,100.0,11/04/2014,facade consulting services,NaN,NaN,NaN,NaN,NaN,"440 PARK AVENUE SOUTH, 15TH FLOOR"
1194,KARTA LEGAL LLC,NaN,Lourdes Fuentes,212-897-9500,212-843-8620,klprojectmanager@kartalegal.com,"MBE,WBE",04/30/2020;04/30/2020,HISPANIC,950 Third Avenue,...,10000.00,100.0,12/01/2017,"E-Discovery, IT, Litigation Support, legal ser...",NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 24TH FLOOR
1241,"SUCCESS-ID, LLC",NaN,Evelyn Herrera,954-600-5494,NaN,customercare@success-id.com,"MBE,WBE",06/30/2024;06/30/2024,HISPANIC,122 Egbert Ave,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122 EGBERT AVE # 2
1784,"MERRICK CAPITAL MANAGEMENT, LLC",NaN,Tammy Jones,212-915-0690,917-591-8781,rcadigan@basisinvgroup.com,MBE,09/30/2022,BLACK,"75 Broad Street, Suite 2110",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"75 BROAD STREET, SUITE 2110"
1840,WOODSIDE HARBOR LLC,NaN,Davin Darnt,718-775-6381,NaN,info@woodsideharbor.com,MBE,06/30/2024,BLACK,58-12 Queens Blvd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58-12 QUEENS BLVD SUITE 3C


In [25]:
nycorp_matched

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent City,Registered Agent State,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,DOS Address
49957,5413824,1733WMOREA&D918 LLC,2018-09-21,NASSAU,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,471 N BROADWAY,#122,,JERICHO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#122
73408,5312228,21 EAST 21ST STREET PARTNERS LLC,2018-03-27,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O TALPION FUND MANAGEMENT LP,950 THIRD AVENUE,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE
107425,5312242,31 EAST 21ST STREET PARTNERS LLC,2018-03-27,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O TALPION FUND MANAGEMENT LP,950 THIRD AVENUE,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE
107526,5424306,31 IRVING COURT LLC,2018-10-11,NASSAU,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,31 IRVING COURT LLC,89 SUMMIT STREET,,OYSTER BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89 SUMMIT STREET
161220,5297227,"57 FRANKLIN STREET, LLC",2018-03-05,ALBANY,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"C/O GRAUBARD & BLUMENFIELD, P.C.",370 LEXINGTON AVENUE,27TH FLOOR,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370 LEXINGTON AVENUE 27TH FLOOR
205059,5437838,95 KANE ST. LLC,2018-11-05,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O BORG LAW,"370 LEXINGTON AVENUE,",SUITE # 800,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"370 LEXINGTON AVENUE, SUITE # 800"
218860,5364766,"ABINGTON GREENVILLE, LLC",2018-06-25,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O ABINGTON HOLDING,950 THIRD AVENUE,27TH FLOOR,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 27TH FLOOR
224947,5355363,ACCESS GENERAL PARTNER LLC,2018-06-07,NEW YORK,DELAWARE,FOREIGN LIMITED LIABILITY COMPANY,ACCESS GENERAL PARTNER LLC,200 WEST STREET,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200 WEST STREET
260068,5288131,AIMS PRIVATE EQUITY PARTICIPANTS XII LLC,2018-02-16,NEW YORK,DELAWARE,FOREIGN LIMITED LIABILITY COMPANY,AIMS PRIVATE EQUITY PARTICIPANTS XII LLC,200 WEST STREET,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200 WEST STREET
302603,5362309,AMBURGEY & ASSOCIATES LLP,2018-06-20,NaN,NEW YORK,DOMESTIC REGISTERED LIMITED LIABILITY PARTNERSHIP,C/O JOHNSON GALLAGHER,75 BROAD STREET,,NEW YORK,...,NaN,NaN,NaN,C/O JOHNSON GALLAGHER,75 BROAD STREET,NaN,NEW YORK,NEW YORK,10004,75 BROAD STREET


### Do fuzzy matching on the new merged columns
*Created a new column 'Similar' in nycorp_matched

In [29]:
nycorp_matched['Similar']=nycorp_matched.loc[:,'DOS Address'].apply(lambda x : [process.extract(x, mwbe_matched.loc[:,'MWBE Address'], limit=1)][0][0][0])

C:\Users\jshyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
nycorp_matched

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent State,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,DOS Address,Similar
49957,5413824,1733WMOREA&D918 LLC,2018-09-21,NASSAU,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,471 N BROADWAY,#122,,JERICHO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#122,122 EGBERT AVE # 2
73408,5312228,21 EAST 21ST STREET PARTNERS LLC,2018-03-27,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O TALPION FUND MANAGEMENT LP,950 THIRD AVENUE,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE,950 THIRD AVENUE 24TH FLOOR
107425,5312242,31 EAST 21ST STREET PARTNERS LLC,2018-03-27,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O TALPION FUND MANAGEMENT LP,950 THIRD AVENUE,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE,950 THIRD AVENUE 24TH FLOOR
107526,5424306,31 IRVING COURT LLC,2018-10-11,NASSAU,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,31 IRVING COURT LLC,89 SUMMIT STREET,,OYSTER BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89 SUMMIT STREET,9 SUMMIT STREET
161220,5297227,"57 FRANKLIN STREET, LLC",2018-03-05,ALBANY,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"C/O GRAUBARD & BLUMENFIELD, P.C.",370 LEXINGTON AVENUE,27TH FLOOR,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370 LEXINGTON AVENUE 27TH FLOOR,370 LEXINGTON AVENUE
205059,5437838,95 KANE ST. LLC,2018-11-05,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O BORG LAW,"370 LEXINGTON AVENUE,",SUITE # 800,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"370 LEXINGTON AVENUE, SUITE # 800",370 LEXINGTON AVENUE
218860,5364766,"ABINGTON GREENVILLE, LLC",2018-06-25,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O ABINGTON HOLDING,950 THIRD AVENUE,27TH FLOOR,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 27TH FLOOR,950 THIRD AVENUE 24TH FLOOR
224947,5355363,ACCESS GENERAL PARTNER LLC,2018-06-07,NEW YORK,DELAWARE,FOREIGN LIMITED LIABILITY COMPANY,ACCESS GENERAL PARTNER LLC,200 WEST STREET,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200 WEST STREET,"200 WEST 54TH STREET, APT. 7J"
260068,5288131,AIMS PRIVATE EQUITY PARTICIPANTS XII LLC,2018-02-16,NEW YORK,DELAWARE,FOREIGN LIMITED LIABILITY COMPANY,AIMS PRIVATE EQUITY PARTICIPANTS XII LLC,200 WEST STREET,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200 WEST STREET,"200 WEST 54TH STREET, APT. 7J"
302603,5362309,AMBURGEY & ASSOCIATES LLP,2018-06-20,NaN,NEW YORK,DOMESTIC REGISTERED LIMITED LIABILITY PARTNERSHIP,C/O JOHNSON GALLAGHER,75 BROAD STREET,,NEW YORK,...,NaN,NaN,C/O JOHNSON GALLAGHER,75 BROAD STREET,NaN,NEW YORK,NEW YORK,10004,75 BROAD STREET,"75 BROAD STREET, SUITE 2110"


### Left Merge 
* mwbe on nycorp for the year 2018

In [54]:
left_merge = pd.merge(nycorp_matched, mwbe_matched, left_on= 'Similar', right_on = 'MWBE Address', how ='left' )

In [55]:
left_merge

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Job_Exp3_Value_of_Contract,Job_Exp3_Percent_Self_Performed,DATEOFJOB3,EXPERIENCE3,Job_Exp4_Name_of_Client,Job_Exp4_Value_of_Contract,Job_Exp4_Percent_Self_Performed,DATEOFJOB4,EXPERIENCE4,MWBE Address
0,5413824,1733WMOREA&D918 LLC,2018-09-21,NASSAU,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,471 N BROADWAY,#122,,JERICHO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122 EGBERT AVE # 2
1,5312228,21 EAST 21ST STREET PARTNERS LLC,2018-03-27,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O TALPION FUND MANAGEMENT LP,950 THIRD AVENUE,,NEW YORK,...,10000.0,100.0,12/01/2017,"E-Discovery, IT, Litigation Support, legal ser...",NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 24TH FLOOR
2,5312242,31 EAST 21ST STREET PARTNERS LLC,2018-03-27,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O TALPION FUND MANAGEMENT LP,950 THIRD AVENUE,,NEW YORK,...,10000.0,100.0,12/01/2017,"E-Discovery, IT, Litigation Support, legal ser...",NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 24TH FLOOR
3,5424306,31 IRVING COURT LLC,2018-10-11,NASSAU,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,31 IRVING COURT LLC,89 SUMMIT STREET,,OYSTER BAY,...,NaN,100.0,08/29/2018,Redesign entrance of residential home includin...,NaN,NaN,NaN,NaN,NaN,9 SUMMIT STREET
4,5297227,"57 FRANKLIN STREET, LLC",2018-03-05,ALBANY,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"C/O GRAUBARD & BLUMENFIELD, P.C.",370 LEXINGTON AVENUE,27TH FLOOR,NEW YORK,...,4450.0,100.0,01/01/2018,Printing,NaN,NaN,NaN,NaN,NaN,370 LEXINGTON AVENUE
5,5437838,95 KANE ST. LLC,2018-11-05,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O BORG LAW,"370 LEXINGTON AVENUE,",SUITE # 800,NEW YORK,...,4450.0,100.0,01/01/2018,Printing,NaN,NaN,NaN,NaN,NaN,370 LEXINGTON AVENUE
6,5364766,"ABINGTON GREENVILLE, LLC",2018-06-25,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O ABINGTON HOLDING,950 THIRD AVENUE,27TH FLOOR,NEW YORK,...,10000.0,100.0,12/01/2017,"E-Discovery, IT, Litigation Support, legal ser...",NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 24TH FLOOR
7,5355363,ACCESS GENERAL PARTNER LLC,2018-06-07,NEW YORK,DELAWARE,FOREIGN LIMITED LIABILITY COMPANY,ACCESS GENERAL PARTNER LLC,200 WEST STREET,,NEW YORK,...,4875.0,100.0,02/04/2018,the client is moving from Australia to open a ...,NaN,NaN,NaN,NaN,NaN,"200 WEST 54TH STREET, APT. 7J"
8,5288131,AIMS PRIVATE EQUITY PARTICIPANTS XII LLC,2018-02-16,NEW YORK,DELAWARE,FOREIGN LIMITED LIABILITY COMPANY,AIMS PRIVATE EQUITY PARTICIPANTS XII LLC,200 WEST STREET,,NEW YORK,...,4875.0,100.0,02/04/2018,the client is moving from Australia to open a ...,NaN,NaN,NaN,NaN,NaN,"200 WEST 54TH STREET, APT. 7J"
9,5362309,AMBURGEY & ASSOCIATES LLP,2018-06-20,NaN,NEW YORK,DOMESTIC REGISTERED LIMITED LIABILITY PARTNERSHIP,C/O JOHNSON GALLAGHER,75 BROAD STREET,,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"75 BROAD STREET, SUITE 2110"


The address from nycorp and mwbe do not match up 100%, need to fuzzy merge on the name

### Full merge

In [31]:
merged_df = nycorp_matched.merge(mwbe_matched, left_on = 'Similar', right_on = 'MWBE Address')

In [36]:
merged_df[['MWBE Address','DOS Address','Current Entity Name','Vendor_Formal_Name']]

,MWBE Address,DOS Address,Current Entity Name,Vendor_Formal_Name
0,122 EGBERT AVE # 2,#122,1733WMOREA&D918 LLC,"SUCCESS-ID, LLC"
1,950 THIRD AVENUE 24TH FLOOR,950 THIRD AVENUE,21 EAST 21ST STREET PARTNERS LLC,KARTA LEGAL LLC
2,950 THIRD AVENUE 24TH FLOOR,950 THIRD AVENUE,31 EAST 21ST STREET PARTNERS LLC,KARTA LEGAL LLC
3,950 THIRD AVENUE 24TH FLOOR,950 THIRD AVENUE 27TH FLOOR,"ABINGTON GREENVILLE, LLC",KARTA LEGAL LLC
4,950 THIRD AVENUE 24TH FLOOR,"950 THIRD AVENUE, 24TH FLOOR",KARTA LEGAL LLC,KARTA LEGAL LLC
5,9 SUMMIT STREET,89 SUMMIT STREET,31 IRVING COURT LLC,"SUE STEELE LANDSCAPE ARCHITECTURE, PLLC"
6,9 SUMMIT STREET,9 SUMMIT STREET,"SUE STEELE LANDSCAPE ARCHITECTURE, PLLC","SUE STEELE LANDSCAPE ARCHITECTURE, PLLC"
7,370 LEXINGTON AVENUE,370 LEXINGTON AVENUE 27TH FLOOR,"57 FRANKLIN STREET, LLC",TRIUMPH PRODUCTIONS INC.
8,370 LEXINGTON AVENUE,"370 LEXINGTON AVENUE, SUITE # 800",95 KANE ST. LLC,TRIUMPH PRODUCTIONS INC.
9,370 LEXINGTON AVENUE,"370 LEXINGTON AVENUE, STE. 908","BROWN'S RIVER MARINA, LLC",TRIUMPH PRODUCTIONS INC.


In [37]:
matched_col = []
for row in merged_df.index:
    cen = merged_df.get_value(row, 'Current Entity Name')
    cm = merged_df.get_value(row, 'Vendor_Formal_Name')
    matched= fuzz.partial_ratio(cen,cm)
    if matched > 95:
        matched_col.append([cen,cm,matched])

C:\Users\jshyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jshyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [38]:
matched_col

[['KARTA LEGAL LLC', 'KARTA LEGAL LLC', 100],
 ['SUE STEELE LANDSCAPE ARCHITECTURE, PLLC',
  'SUE STEELE LANDSCAPE ARCHITECTURE, PLLC',
  100],
 ["DON'T FAH-GET HOME IMPROVEMENTS LLC",
  "DON'T FAH-GET HOME IMPROVEMENTS LLC",
  100],
 ['CRESCENT TECHNOLOGY SYSTEM INTERGRATORS INC',
  'CRESCENT TECHNOLOGY SYSTEM INTERGRATORS INC',
  100],
 ['HANDS OF VISIONS ARTWORK INC', 'HANDS OF VISIONS ARTWORK', 100],
 ['EXCEED FLOORING INC', 'EXCEED FLOORING INC', 100],
 ['HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.',
  'HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.',
  100],
 ['KS4INSPIRATION LLC', 'KS4INSPIRATION LLC', 100],
 ['LAW OFFICE OF F. RICHARD HURLEY, P.C.',
  'THE LAW OFFICE OF F. RICHARD HURLEY, P.C.',
  100],
 ['LUCCAH CONSULTING LLC', 'LUCCAH CONSULTING LLC', 100],
 ['RICHTER GEOLOGY PLLC', 'RICHTER GEOLOGY PLLC', 100],
 ['ROYLE I.T. SERVICES, LLC', 'ROYLE I.T. SERVICE', 100],
 ['SKASA TECHNOLOGIES INC', 'SKASA TECHNOLOGIES INC', 100],
 ['URBAN STREAM MEDIA LLC', 'URBAN STREAM 

In [42]:
df_2018 = merged_df[merged_df['Current Entity Name'].isin([col[0] for col in matched_col])]

In [43]:
df_2018[['MWBE Address', 'Vendor_Formal_Name','DOS Address', 'Current Entity Name']]

,MWBE Address,Vendor_Formal_Name,DOS Address,Current Entity Name
4,950 THIRD AVENUE 24TH FLOOR,KARTA LEGAL LLC,"950 THIRD AVENUE, 24TH FLOOR",KARTA LEGAL LLC
6,9 SUMMIT STREET,"SUE STEELE LANDSCAPE ARCHITECTURE, PLLC",9 SUMMIT STREET,"SUE STEELE LANDSCAPE ARCHITECTURE, PLLC"
40,44 COURT STREET STE 1217,DON'T FAH-GET HOME IMPROVEMENTS LLC,44 COURT STREET SUITE 1217,DON'T FAH-GET HOME IMPROVEMENTS LLC
53,626 MERRICK ROAD,CRESCENT TECHNOLOGY SYSTEM INTERGRATORS INC,626 MERRICK ROAD,CRESCENT TECHNOLOGY SYSTEM INTERGRATORS INC
60,10460 QUEENS BLVD. APT# 3G,HANDS OF VISIONS ARTWORK,10460 QUEENS BLVD APT 3G,HANDS OF VISIONS ARTWORK INC
63,74 IRMA DRIVE,EXCEED FLOORING INC,74 IRMA DRIVE,EXCEED FLOORING INC
70,"440 PARK AVENUE SOUTH, 15TH FLOOR",HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.,440 PARK AVENUE SOUTH 15TH FLOOR,HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.
72,1841 CENTRAL PARK AVE APT. 8D,KS4INSPIRATION LLC,1841 CENTRAL PARK AVE APT 8D,KS4INSPIRATION LLC
73,30 VESEY STREET 15TH FLOOR,"THE LAW OFFICE OF F. RICHARD HURLEY, P.C.",30 VESEY STREET 15TH FLOOR,"LAW OFFICE OF F. RICHARD HURLEY, P.C."
75,25 BROAD STREET SUITE 21F,LUCCAH CONSULTING LLC,25 BROAD STREET SUITE 21F,LUCCAH CONSULTING LLC


In [44]:
df_2018.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Job_Exp3_Value_of_Contract,Job_Exp3_Percent_Self_Performed,DATEOFJOB3,EXPERIENCE3,Job_Exp4_Name_of_Client,Job_Exp4_Value_of_Contract,Job_Exp4_Percent_Self_Performed,DATEOFJOB4,EXPERIENCE4,MWBE Address
4,5390236,KARTA LEGAL LLC,2018-08-09,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,KARTA LEGAL LLC,"950 THIRD AVENUE, 24TH FLOOR",,NEW YORK,...,10000.0,100.0,12/01/2017,"E-Discovery, IT, Litigation Support, legal ser...",NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 24TH FLOOR
6,5260276,"SUE STEELE LANDSCAPE ARCHITECTURE, PLLC",2018-01-03,MONROE,NEW YORK,DOMESTIC PROFESSIONAL SERVICE LIMITED LIABILIT...,"SUE STEELE LANDSCAPE ARCHITECTURE, PLLC",9 SUMMIT STREET,,FAIRPORT,...,NaN,100.0,08/29/2018,Redesign entrance of residential home includin...,NaN,NaN,NaN,NaN,NaN,9 SUMMIT STREET
40,5267490,DON'T FAH-GET HOME IMPROVEMENTS LLC,2018-01-16,KINGS,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,DON'T FAH-GET HOME IMPROVEMENTS LLC,44 COURT STREET SUITE 1217,,BROOKLYN,...,10000.0,100.0,09/17/2018,Waterproofing the Roof,NaN,NaN,NaN,NaN,NaN,44 COURT STREET STE 1217
53,5260322,CRESCENT TECHNOLOGY SYSTEM INTERGRATORS INC,2018-01-03,NASSAU,NEW YORK,DOMESTIC BUSINESS CORPORATION,SYED NASIR,626 MERRICK ROAD,,LYNBROOK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,626 MERRICK ROAD
60,5289096,HANDS OF VISIONS ARTWORK INC,2018-02-20,QUEENS,NEW YORK,DOMESTIC BUSINESS CORPORATION,ROBIN MEEKS,10460 QUEENS BLVD APT 3G,,FOREST HILLS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10460 QUEENS BLVD. APT# 3G


In [56]:
left = df_2018 = left_merge[left_merge['Current Entity Name'].isin([col[0] for col in matched_col])]

In [57]:
left

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Job_Exp3_Value_of_Contract,Job_Exp3_Percent_Self_Performed,DATEOFJOB3,EXPERIENCE3,Job_Exp4_Name_of_Client,Job_Exp4_Value_of_Contract,Job_Exp4_Percent_Self_Performed,DATEOFJOB4,EXPERIENCE4,MWBE Address
21,5260322,CRESCENT TECHNOLOGY SYSTEM INTERGRATORS INC,2018-01-03,NASSAU,NEW YORK,DOMESTIC BUSINESS CORPORATION,SYED NASIR,626 MERRICK ROAD,,LYNBROOK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,626 MERRICK ROAD
27,5267490,DON'T FAH-GET HOME IMPROVEMENTS LLC,2018-01-16,KINGS,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,DON'T FAH-GET HOME IMPROVEMENTS LLC,44 COURT STREET SUITE 1217,,BROOKLYN,...,10000.0,100.0,09/17/2018,Waterproofing the Roof,NaN,NaN,NaN,NaN,NaN,44 COURT STREET STE 1217
34,5322772,EXCEED FLOORING INC,2018-04-13,NASSAU,NEW YORK,DOMESTIC BUSINESS CORPORATION,EXCEED FLOORING INC,74 IRMA DRIVE,,OCEANSIDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 IRMA DRIVE
42,5289096,HANDS OF VISIONS ARTWORK INC,2018-02-20,QUEENS,NEW YORK,DOMESTIC BUSINESS CORPORATION,ROBIN MEEKS,10460 QUEENS BLVD APT 3G,,FOREST HILLS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10460 QUEENS BLVD. APT# 3G
44,5259100,HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.,2018-01-02,NEW YORK,NEW YORK,DOMESTIC PROFESSIONAL CORPORATION,HEINTGES CONSULTING ARCHITECTS & ENGINEERS P.C.,440 PARK AVENUE SOUTH,15TH FLOOR,NEW YORK,...,0.0,100.0,11/04/2014,facade consulting services,NaN,NaN,NaN,NaN,NaN,"440 PARK AVENUE SOUTH, 15TH FLOOR"
47,5390236,KARTA LEGAL LLC,2018-08-09,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,KARTA LEGAL LLC,"950 THIRD AVENUE, 24TH FLOOR",,NEW YORK,...,10000.0,100.0,12/01/2017,"E-Discovery, IT, Litigation Support, legal ser...",NaN,NaN,NaN,NaN,NaN,950 THIRD AVENUE 24TH FLOOR
50,5378125,KS4INSPIRATION LLC,2018-07-18,WESTCHESTER,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,KEVIN SAUNDERS,1841 CENTRAL PARK AVE APT 8D,,YONKERS,...,75.0,100.0,02/16/2018,Motivational speech,"Faculty Student Association of WCC, Inc.",300.0,100.0,01/31/2018,Motivational workshop,1841 CENTRAL PARK AVE APT. 8D
51,5398465,"LAW OFFICE OF F. RICHARD HURLEY, P.C.",2018-08-23,NEW YORK,NEW YORK,DOMESTIC PROFESSIONAL CORPORATION,"LAW OFFICE OF F. RICHARD HURLEY, P.C.",30 VESEY STREET,15TH FLOOR,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 VESEY STREET 15TH FLOOR
55,5395080,LUCCAH CONSULTING LLC,2018-08-17,NEW YORK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,AJLA ANTONUCCI,25 BROAD STREET,SUITE 21F,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25 BROAD STREET SUITE 21F
72,5334502,RICHTER GEOLOGY PLLC,2018-05-03,ALBANY,NEW HAMPSHIRE,FOREIGN PROFESSIONAL SERVICE LIMITED LIABILITY...,RICHTER GEOLOGY PLLC,140 RANGE RD,,WINDHAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140 RANGE RD
